In [1]:
## UofT SCS FinTech Bootcamp Project 1 Team Code
## Project team: Ambreen Simon, George Barbuc, Katie Gouinlock, Keiran Gugula
# Import libraries required for analysis and visualization
from pathlib import Path
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime as dt
import yfinance as yf
import random

%matplotlib inline

In [2]:
# Create dataframe with S&P 500 tickers based on information in CSV file
sp_csv = pd.read_csv(Path("sp500.csv"))

In [3]:
# Keiran can you confirm what this step does ?
tickers = [x for x in sp_csv.loc[:, "Symbol"]]

# Appending SPY ticker to S&P 500 consitutents. SPY will be used as a proxy for S&P 500 returns
tickers.append("SPY")

# Downloading historical price data for S&P 500 securities using Yfinance library
data = yf.download(tickers=tickers, start="2012-01-01", end="2017-01-01", group_by="ticker")

[*********************100%***********************]  506 of 506 completed

10 Failed downloads:
- OTIS: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- BRK.B: No data found, symbol may be delisted
- FOX: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- DOW: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- FOXA: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- CTVA: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- IR: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- VIAC: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- CARR: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- BF.B: No data found for this date range, symbol may be delisted


In [47]:
data.head()

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,BF.B,KO,MMM
,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,...,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,38.310001,45.490002,15.78,47.720001,9.44,12.12,39.912636,18.639999,57.790001,40.520000,...,47.813873,36.732372,41.799999,33.976665,16.770000,22.645000,37.709999,NaN,35.070000,83.489998
2012-01-04,38.849998,45.480000,15.75,48.029999,9.20,12.03,40.189743,19.010000,57.650002,40.880001,...,47.761192,36.664139,40.709999,34.873333,17.139999,23.084999,37.009998,NaN,34.849998,84.180000
2012-01-05,39.500000,45.660000,15.80,47.720001,8.92,12.32,40.618816,19.110001,57.480000,40.950001,...,48.112381,36.838512,40.299999,34.419998,17.620001,23.180000,38.060001,NaN,34.685001,83.800003
2012-01-06,39.910000,45.290001,15.78,47.150002,8.83,12.23,39.635525,19.120001,58.009998,41.169998,...,48.182617,36.641396,39.880001,34.163334,17.639999,23.360001,39.200001,NaN,34.465000,83.370003
2012-01-09,39.750000,45.540001,15.67,47.889999,8.84,12.47,39.349480,19.070000,58.029999,41.080002,...,47.998245,36.830933,39.810001,34.556667,17.879999,23.295000,39.040001,NaN,34.465000,83.870003


In [48]:
# Creating function in order to clean data downloaded from yfinance library
# Cleaning data requires dropping all columns other than close price, and then returning the resulting data frame
def clean_yahoo_df(data):
    data.drop(["Open", "High", "Low", "Adj Close", "Volume"], axis=1, level=1, inplace=True)
    df_clean = data.droplevel(axis=1, level=1)
    return df_clean
    
sp_df_clean = clean_yahoo_df(data)
sp_df_clean.head()

KeyError: "labels ['Open' 'High' 'Low' 'Adj Close' 'Volume'] not found in level"

In [49]:
# Storing cleaned data from yfinance to a new dataframe
sp_df_clean.to_csv("sp500_raw_data.csv")

In [50]:
# Creating a new dataframe from the cleaned yfinance S&P 500 data, setting index as the date
sp_path = Path("sp500_raw_data.csv")
sp_combined_df = pd.read_csv(sp_path, index_col='Date',infer_datetime_format=True,parse_dates=True)
sp_combined_df.head(5)

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,BF.B,KO,MMM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,38.310001,45.490002,15.78,47.720001,9.44,12.12,39.912636,18.639999,57.790001,40.520000,...,47.813873,36.732372,41.799999,33.976665,16.770000,22.645000,37.709999,NaN,35.070000,83.489998
2012-01-04,38.849998,45.480000,15.75,48.029999,9.20,12.03,40.189743,19.010000,57.650002,40.880001,...,47.761192,36.664139,40.709999,34.873333,17.139999,23.084999,37.009998,NaN,34.849998,84.180000
2012-01-05,39.500000,45.660000,15.80,47.720001,8.92,12.32,40.618816,19.110001,57.480000,40.950001,...,48.112381,36.838512,40.299999,34.419998,17.620001,23.180000,38.060001,NaN,34.685001,83.800003
2012-01-06,39.910000,45.290001,15.78,47.150002,8.83,12.23,39.635525,19.120001,58.009998,41.169998,...,48.182617,36.641396,39.880001,34.163334,17.639999,23.360001,39.200001,NaN,34.465000,83.370003
2012-01-09,39.750000,45.540001,15.67,47.889999,8.84,12.47,39.349480,19.070000,58.029999,41.080002,...,47.998245,36.830933,39.810001,34.556667,17.879999,23.295000,39.040001,NaN,34.465000,83.870003


In [51]:
## Commencing data cleaning procedures
# Check to confirm whether there are any duplicates in the index variable
sp_combined_df.duplicated()

Date
2012-01-03    False
2012-01-04    False
2012-01-05    False
2012-01-06    False
2012-01-09    False
              ...  
2016-12-23    False
2016-12-27    False
2016-12-28    False
2016-12-29    False
2016-12-30    False
Length: 1260, dtype: bool

In [52]:
# Writing data to a csv file for future use
sp_combined_df.to_csv("sp500_clean.csv")

In [53]:
# Reading in data from csv file and continuing cleaning procedures, beginning first with visualizing data
sp_csv_clean = pd.read_csv(Path("sp500_clean.csv"),index_col='Date',infer_datetime_format=True,parse_dates=True)
sp_csv_clean.head()

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,BF.B,KO,MMM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,38.310001,45.490002,15.78,47.720001,9.44,12.12,39.912636,18.639999,57.790001,40.520000,...,47.813873,36.732372,41.799999,33.976665,16.770000,22.645000,37.709999,NaN,35.070000,83.489998
2012-01-04,38.849998,45.480000,15.75,48.029999,9.20,12.03,40.189743,19.010000,57.650002,40.880001,...,47.761192,36.664139,40.709999,34.873333,17.139999,23.084999,37.009998,NaN,34.849998,84.180000
2012-01-05,39.500000,45.660000,15.80,47.720001,8.92,12.32,40.618816,19.110001,57.480000,40.950001,...,48.112381,36.838512,40.299999,34.419998,17.620001,23.180000,38.060001,NaN,34.685001,83.800003
2012-01-06,39.910000,45.290001,15.78,47.150002,8.83,12.23,39.635525,19.120001,58.009998,41.169998,...,48.182617,36.641396,39.880001,34.163334,17.639999,23.360001,39.200001,NaN,34.465000,83.370003
2012-01-09,39.750000,45.540001,15.67,47.889999,8.84,12.47,39.349480,19.070000,58.029999,41.080002,...,47.998245,36.830933,39.810001,34.556667,17.879999,23.295000,39.040001,NaN,34.465000,83.870003


In [54]:
# Confirming number of entries in data frame
sp_csv_clean.shape

(1260, 506)

In [55]:
# Checking for null values
sp_csv_clean.isnull().sum()

DIS        2
ETN        2
NLOK       2
EMR        2
DXCM       2
        ... 
SBUX       2
UHS        2
BF.B    1260
KO         2
MMM        2
Length: 506, dtype: int64

In [56]:
# Further exploring data frame in preparation for cleaning of data
sp_csv_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1260 entries, 2012-01-03 to 2016-12-30
Columns: 506 entries, DIS to MMM
dtypes: float64(506)
memory usage: 4.9 MB


In [57]:
# Confirming data types in dataframe are all float values
sp_csv_clean.dtypes

DIS     float64
ETN     float64
NLOK    float64
EMR     float64
DXCM    float64
         ...   
SBUX    float64
UHS     float64
BF.B    float64
KO      float64
MMM     float64
Length: 506, dtype: object

In [58]:
# Confirming maximum amount of data, i.e. number of trading days
# that will be included in the period is 1258
sp_csv_clean.count()

DIS     1258
ETN     1258
NLOK    1258
EMR     1258
DXCM    1258
        ... 
SBUX    1258
UHS     1258
BF.B       0
KO      1258
MMM     1258
Length: 506, dtype: int64

In [59]:
# Checking for null values in data set (anticipate there will be some)
# with stocks that were not present in the S&P 500 for the full time, as well as 2 days of blank trading data previously identified

sp_csv_clean.isnull()

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,BF.B,KO,MMM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2012-01-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2012-01-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2012-01-06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2012-01-09,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2016-12-27,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2016-12-28,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [60]:
#Checking for nulls
sp_csv_clean.isnull().mean() *100

DIS       0.15873
ETN       0.15873
NLOK      0.15873
EMR       0.15873
DXCM      0.15873
          ...    
SBUX      0.15873
UHS       0.15873
BF.B    100.00000
KO        0.15873
MMM       0.15873
Length: 506, dtype: float64

In [61]:
# Dropping all rows with morethan two null values, threshold prevents us from dropping entire data set as a result of two trading days of missning data
df_all_no_null=sp_csv_clean.dropna(thresh=2).copy()

In [62]:
# Drop those tickers that do not have complete price history for the 2012 to 2017 time fram in scope for analysis
df_all_clean = df_all_no_null.dropna(axis=1)
df_all_clean.head()

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,KR,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,KO,MMM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,38.310001,45.490002,15.78,47.720001,9.44,12.12,39.912636,18.639999,57.790001,40.520000,...,12.260,47.813873,36.732372,41.799999,33.976665,16.770000,22.645000,37.709999,35.070000,83.489998
2012-01-04,38.849998,45.480000,15.75,48.029999,9.20,12.03,40.189743,19.010000,57.650002,40.880001,...,12.135,47.761192,36.664139,40.709999,34.873333,17.139999,23.084999,37.009998,34.849998,84.180000
2012-01-05,39.500000,45.660000,15.80,47.720001,8.92,12.32,40.618816,19.110001,57.480000,40.950001,...,12.145,48.112381,36.838512,40.299999,34.419998,17.620001,23.180000,38.060001,34.685001,83.800003
2012-01-06,39.910000,45.290001,15.78,47.150002,8.83,12.23,39.635525,19.120001,58.009998,41.169998,...,12.095,48.182617,36.641396,39.880001,34.163334,17.639999,23.360001,39.200001,34.465000,83.370003
2012-01-09,39.750000,45.540001,15.67,47.889999,8.84,12.47,39.349480,19.070000,58.029999,41.080002,...,12.130,47.998245,36.830933,39.810001,34.556667,17.879999,23.295000,39.040001,34.465000,83.870003


In [63]:
# Confirm cleanse of data worked by re-running check for null values
df_all_no_null.isnull().sum()

DIS        0
ETN        0
NLOK       0
EMR        0
DXCM       0
        ... 
SBUX       0
UHS        0
BF.B    1258
KO         0
MMM        0
Length: 506, dtype: int64

In [64]:
df_all_no_null.head()

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,BF.B,KO,MMM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,38.310001,45.490002,15.78,47.720001,9.44,12.12,39.912636,18.639999,57.790001,40.520000,...,47.813873,36.732372,41.799999,33.976665,16.770000,22.645000,37.709999,NaN,35.070000,83.489998
2012-01-04,38.849998,45.480000,15.75,48.029999,9.20,12.03,40.189743,19.010000,57.650002,40.880001,...,47.761192,36.664139,40.709999,34.873333,17.139999,23.084999,37.009998,NaN,34.849998,84.180000
2012-01-05,39.500000,45.660000,15.80,47.720001,8.92,12.32,40.618816,19.110001,57.480000,40.950001,...,48.112381,36.838512,40.299999,34.419998,17.620001,23.180000,38.060001,NaN,34.685001,83.800003
2012-01-06,39.910000,45.290001,15.78,47.150002,8.83,12.23,39.635525,19.120001,58.009998,41.169998,...,48.182617,36.641396,39.880001,34.163334,17.639999,23.360001,39.200001,NaN,34.465000,83.370003
2012-01-09,39.750000,45.540001,15.67,47.889999,8.84,12.47,39.349480,19.070000,58.029999,41.080002,...,47.998245,36.830933,39.810001,34.556667,17.879999,23.295000,39.040001,NaN,34.465000,83.870003


In [65]:
# Save cleaned data to a new csv for further analysis
df_all_clean.to_csv("final_clean_df.csv")

In [66]:
# Create new dataframe with daily returns based on S&P 500 price data
daily_returns = df_all_clean.pct_change()

# Drop first row which is null following conversion to returns
daily_returns.dropna().copy()
daily_returns

,DIS,ETN,NLOK,EMR,DXCM,AES,DRI,TXT,DGX,NUE,...,KR,ADP,DHR,LLY,FLS,ZION,SBUX,UHS,KO,MMM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,0.014095,-0.000220,-0.001901,0.006496,-0.025424,-0.007426,0.006943,0.019850,-0.002423,0.008885,...,-0.010196,-0.001102,-0.001858,-0.026077,0.026391,0.022063,0.019430,-0.018563,-0.006273,0.008264
2012-01-05,0.016731,0.003958,0.003175,-0.006454,-0.030435,0.024106,0.010676,0.005260,-0.002949,0.001712,...,0.000824,0.007353,0.004756,-0.010071,-0.012999,0.028005,0.004115,0.028371,-0.004734,-0.004514
2012-01-06,0.010380,-0.008103,-0.001266,-0.011945,-0.010090,-0.007305,-0.024208,0.000523,0.009221,0.005372,...,-0.004117,0.001460,-0.005351,-0.010422,-0.007457,0.001135,0.007765,0.029953,-0.006343,-0.005131
2012-01-09,-0.004009,0.005520,-0.006971,0.015695,0.001133,0.019624,-0.007217,-0.002615,0.000345,-0.002186,...,0.002894,-0.003827,0.005173,-0.001755,0.011513,0.013605,-0.002783,-0.004082,0.000000,0.005997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,-0.002561,0.002791,0.001244,0.001940,0.006602,-0.004244,-0.001207,0.007203,0.006535,-0.002272,...,0.002859,0.003216,0.007915,0.007234,-0.006916,0.002743,-0.001751,0.010391,0.001203,-0.002511
2016-12-27,0.000190,0.003369,0.009114,0.006514,-0.001760,0.011083,-0.001477,0.002043,0.001082,0.006180,...,0.002565,0.003692,-0.001647,0.000271,0.012700,0.001596,-0.002631,-0.007042,0.000240,0.000951
2016-12-28,-0.008272,-0.011535,-0.008621,-0.015218,-0.022276,-0.018550,-0.009278,-0.010604,-0.008107,-0.015355,...,-0.013932,-0.002517,-0.008754,-0.010161,-0.018811,-0.014796,-0.008969,-0.009611,-0.005287,-0.004695


In [67]:
# Create function find_beta to calculate beta for each ticker and store the result in a dictionary

def find_beta(df):

    index_returns = df.iloc[:,0]
    df_beta = {}
    
    for ticker in df:
        covariance = df[ticker].cov(index_returns)
        variance = df[ticker].var()
        beta = covariance/variance
        df_beta[ticker] = beta
    
    #new_df["beta"] = 
    
    return df_beta



In [68]:
# Calculate and store betas in a dictionary for each ticker in the daily returns data (S&P 500 + SPY)
beta = find_beta(daily_returns)


In [69]:
# Create df with beta values, sort in descending order, and print output to validate calculations
beta_df = pd.DataFrame.from_dict(beta, orient="index")
beta_df = beta_df.rename(columns={0: "Beta"})
beta_df = beta_df.sort_values(["Beta"],ascending=False)
beta_df.tail()

,Beta
VRTX,0.084184
AMD,0.082526
ABMD,0.071854
NFLX,0.056312
NEM,0.005846


In [70]:
# Start construction of high and low vol portfolios, by transposting daily returns data and sorting by t
daily_returns_transposed = daily_returns.transpose()
daily_returns_transposed.sort_index(ascending=True)

Date,2012-01-03,2012-01-04,2012-01-05,2012-01-06,2012-01-09,2012-01-10,2012-01-11,2012-01-12,2012-01-13,2012-01-17,...,2016-12-16,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-27,2016-12-28,2016-12-29,2016-12-30
A,NaN,-0.007950,0.022382,0.010811,0.026203,0.018760,-0.004604,0.015416,-0.011640,0.023810,...,-0.013961,-0.010455,0.017169,-0.003679,-0.001520,0.005003,0.006710,-0.016986,-0.001750,-0.001753
AAL,NaN,-0.017578,0.087475,0.023766,0.021429,0.000000,0.033217,0.023689,-0.013223,-0.015075,...,-0.013872,0.019945,0.013174,0.005079,-0.015767,-0.004313,0.002682,-0.019338,-0.008811,-0.011852
AAP,NaN,0.023444,0.006363,0.004637,-0.000280,0.000839,0.006709,-0.005137,-0.002652,0.008676,...,-0.005841,0.011110,0.010413,-0.012071,-0.017982,0.002934,0.005559,-0.008263,-0.000821,-0.006812
AAPL,NaN,0.005374,0.011102,0.010454,-0.001586,0.003580,-0.001630,-0.002745,-0.003749,0.011648,...,0.001295,0.005777,0.002658,0.000941,-0.006578,0.001978,0.006351,-0.004264,-0.000257,-0.007796
ABC,NaN,0.003415,0.007592,0.002598,-0.000518,0.011408,-0.010254,0.005957,-0.026777,0.013228,...,0.009260,0.013440,0.011222,-0.005675,-0.013190,0.009639,0.003310,-0.005963,0.012636,-0.014495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,NaN,0.033015,-0.014247,-0.016797,0.003973,0.012267,0.018765,-0.013814,-0.002724,0.000000,...,-0.000798,0.013174,0.003152,-0.001767,0.005902,-0.002347,-0.002941,-0.018089,-0.003404,-0.005023
YUM,NaN,0.006829,0.007631,0.007237,-0.003843,0.006206,0.015503,-0.000985,0.006080,0.010779,...,0.006895,-0.003268,0.002811,-0.000467,-0.002181,-0.003747,0.003448,-0.004841,-0.001412,-0.004715
ZBH,NaN,-0.013534,0.011840,0.002600,0.007410,0.020964,-0.003963,-0.004340,-0.000363,0.002725,...,0.008281,-0.006184,-0.009819,-0.001865,-0.000885,0.007384,0.008014,-0.000776,0.004269,-0.002899
ZBRA,NaN,-0.007559,-0.001410,-0.008192,-0.004557,0.009156,0.009073,-0.016859,-0.000572,0.016586,...,-0.007981,0.020823,0.014024,-0.008230,-0.006915,0.002901,0.007290,-0.016542,0.003855,-0.002094


In [71]:
# Confirm shape including # of rows (and therefore unique tickers) in daily returns dataframe
daily_returns_transposed.shape

(465, 1258)

In [72]:
# Sort dataframe by ticker and confirm # of rows (and therefore unique tickers)
beta_df = beta_df.sort_index(ascending=True)
beta_df.head()

,Beta
A,0.341602
AAL,0.139104
AAP,0.199245
AAPL,0.209772
ABC,0.278930


In [73]:
# Merge beta information with daily returns in order to assign a calculated beta to each of the stock tickers in the dataset
df_returns_w_beta = pd.concat([beta_df, daily_returns_transposed], join="inner", axis=1)
# Sort values from low to high beta in preparation for creating low and high beta portfolios
df_returns_w_beta = df_returns_w_beta.sort_values(["Beta"])
df_returns_w_beta

,Beta,2012-01-03 00:00:00,2012-01-04 00:00:00,2012-01-05 00:00:00,2012-01-06 00:00:00,2012-01-09 00:00:00,2012-01-10 00:00:00,2012-01-11 00:00:00,2012-01-12 00:00:00,2012-01-13 00:00:00,...,2016-12-16 00:00:00,2016-12-19 00:00:00,2016-12-20 00:00:00,2016-12-21 00:00:00,2016-12-22 00:00:00,2016-12-23 00:00:00,2016-12-27 00:00:00,2016-12-28 00:00:00,2016-12-29 00:00:00,2016-12-30 00:00:00
NEM,0.005846,NaN,-0.003061,0.003393,-0.002093,-0.007907,0.019031,0.011014,0.011051,-0.010150,...,0.024264,0.003790,-0.003776,-0.003474,-0.003486,0.032443,0.004005,0.005830,0.075961,-0.034023
NFLX,0.056312,NaN,0.113649,-0.014295,0.088146,0.137791,-0.023936,-0.038401,0.000000,0.024200,...,-0.006240,0.009902,-0.002630,0.011029,-0.007273,0.000080,0.021976,-0.019166,-0.004448,-0.012208
ABMD,0.071854,NaN,-0.007671,0.008835,-0.007116,0.003859,-0.002746,0.004405,0.004386,-0.018559,...,-0.009522,-0.006551,-0.031429,0.004067,-0.017700,0.007441,0.012102,-0.008440,0.007093,-0.007924
AMD,0.082526,NaN,-0.003650,0.000000,-0.005495,0.029466,0.021467,0.017513,0.001721,-0.027491,...,-0.018416,0.027204,0.051142,-0.003475,0.011334,-0.001724,0.042314,-0.043082,0.003463,-0.021570
VRTX,0.084184,NaN,-0.000931,0.028571,0.028080,0.047871,0.017937,0.027533,-0.006699,-0.007014,...,-0.003245,-0.006510,0.001835,-0.050236,-0.007989,0.031797,0.011842,-0.007980,-0.002011,-0.010344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AJG,0.619353,NaN,-0.015921,0.002442,0.002132,0.001519,0.001517,0.000303,0.000606,-0.001816,...,0.014364,0.002328,0.003871,-0.002699,0.004833,0.000577,-0.001154,-0.006352,0.004262,0.002315
MMC,0.638887,NaN,-0.011314,-0.009854,-0.013804,0.002279,0.007795,0.005156,0.005771,0.000000,...,-0.001459,0.003214,-0.004222,-0.001170,0.001464,0.001608,-0.004670,-0.007038,0.000443,-0.002362
ADP,0.649391,NaN,-0.001102,0.007353,0.001460,-0.003827,-0.000183,-0.000915,0.006409,-0.000182,...,0.021162,0.002380,0.012266,0.001368,0.001269,0.003216,0.003692,-0.002517,0.003882,-0.006477
SPY,0.980462,NaN,0.001569,0.002662,-0.002577,0.002427,0.008671,0.000542,0.002399,-0.005173,...,-0.007804,0.002177,0.003858,-0.002783,-0.001727,0.001464,0.002481,-0.008265,-0.000223,-0.003655


In [74]:
# Creating two portfolios, one with stocks with the highest 50 betas, the other with the lowest 50 betas
low_vol_port = df_returns_w_beta.head(50)
high_vol_port = df_returns_w_beta.tail(50)

In [75]:
# Once portfolio has been selected, drop beta value, as it is no longer required in order to perform portfolio analysis
low_vol_no_beta = low_vol_port.drop(columns=["Beta"])
high_vol_no_beta = high_vol_port.drop(columns=["Beta"])

In [76]:
# Re-orienting portfolio dataframes in order to have data as a column and tickers as rows
low_vol = low_vol_no_beta.transpose()
high_vol = high_vol_no_beta.transpose()

In [77]:
# Dropping first row of null values (Jan 3 data) to create final high and low beta porfolios
low_vol.dropna(inplace=True)
high_vol.dropna(inplace=True)

In [78]:
# Creating a portfolio with just "SPY US EQUITY" which will function as a proxy for the S&P 500 index return
index_port = daily_returns["SPY"]
index_port.dropna(inplace=True)
index_port.head()

Date
2012-01-04    0.001569
2012-01-05    0.002662
2012-01-06   -0.002577
2012-01-09    0.002427
2012-01-10    0.008671
Name: SPY, dtype: float64

In [79]:
# Generating fourth portfolio of randomly chosen symbols
rand_port = daily_returns.sample(n=50,axis='columns')
rand_port.dropna(inplace=True)